In [2]:
!pip install wandb -qU

     |████████████████████████████████| 1.9 MB 5.2 MB/s 
     |████████████████████████████████| 162 kB 48.8 MB/s 
     |████████████████████████████████| 182 kB 43.5 MB/s 
     |████████████████████████████████| 63 kB 1.3 MB/s 
     |████████████████████████████████| 162 kB 51.5 MB/s 
     |████████████████████████████████| 158 kB 51.7 MB/s 
     |████████████████████████████████| 157 kB 62.6 MB/s 
     |████████████████████████████████| 157 kB 50.5 MB/s 
     |████████████████████████████████| 157 kB 58.6 MB/s 
     |████████████████████████████████| 157 kB 48.7 MB/s 
     |████████████████████████████████| 157 kB 49.0 MB/s 
     |████████████████████████████████| 157 kB 42.0 MB/s 
     |████████████████████████████████| 157 kB 59.4 MB/s 
     |████████████████████████████████| 156 kB 46.4 MB/s 


In [3]:
import wandb
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
# import the necessary packages
from imutils import paths
import logging
import os
import cv2
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [5]:
# configure logging
# reference for a logging obj
logger = logging.getLogger()

# set level of logging
logger.setLevel(logging.INFO)

# create handlers
c_handler = logging.StreamHandler()
c_format = logging.Formatter(fmt="%(asctime)s %(message)s",datefmt='%d-%m-%Y %H:%M:%S')
c_handler.setFormatter(c_format)

# add handler to the logger
logger.handlers[0] = c_handler

In [6]:
# since we are using Jupyter Notebooks we can replace our argument
# parsing code with *hard coded* arguments and values
args = {
  "project_name": "first_image_classifier",
  "artifact_name_feature": "clean_data:latest",
  "artifact_name_target": "label:latest",
  "train_feature_artifact": "train_x",
  "train_target_artifact": "train_y",
  "test_feature_artifact": "test_x",
  "test_target_artifact": "test_y"
}

In [7]:
# open the W&B project created in the Fetch step
run = wandb.init(entity="pablosanttana",project=args["project_name"], job_type="data_segregation")

logger.info("Downloading and reading clean data artifact")
clean_data = run.use_artifact(args["artifact_name_feature"])
clean_data_path = clean_data.file()

logger.info("Downloading and reading label data artifact")
label_data = run.use_artifact(args["artifact_name_target"])
label_data_path = label_data.file()

# unpacking the artifacts
data = joblib.load(clean_data_path)
label = joblib.load(label_data_path)

wandb: Currently logged in as: pablosanttana. Use `wandb login --relogin` to force relogin


16-10-2022 15:09:12 Downloading and reading clean data artifact
16-10-2022 15:09:13 Downloading and reading label data artifact


In [8]:
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(train_x, test_x, train_y, test_y) = train_test_split(data, label,test_size=0.25, random_state=42)

In [9]:
logger.info("Train x: {}".format(train_x.shape))
logger.info("Train y: {}".format(train_y.shape))
logger.info("Test x: {}".format(test_x.shape))
logger.info("Test y: {}".format(test_y.shape))

16-10-2022 15:09:39 Train x: (2250, 3072)
16-10-2022 15:09:39 Train y: (2250,)
16-10-2022 15:09:39 Test x: (750, 3072)
16-10-2022 15:09:39 Test y: (750,)


In [10]:
logger.info("Dumping the train and test data artifacts to the disk")

# Save the artifacts using joblib
joblib.dump(train_x, args["train_feature_artifact"])
joblib.dump(train_y, args["train_target_artifact"])
joblib.dump(test_x, args["test_feature_artifact"])
joblib.dump(test_y, args["test_target_artifact"])

16-10-2022 15:09:49 Dumping the train and test data artifacts to the disk


['test_y']

In [11]:
# train_x artifact
artifact = wandb.Artifact(args["train_feature_artifact"],
                          type="TRAIN_DATA",
                          description="A json file representing the train_x"
                          )

logger.info("Logging train_x artifact")
artifact.add_file(args["train_feature_artifact"])
run.log_artifact(artifact)

16-10-2022 15:09:59 Logging train_x artifact


In [12]:
# train_y artifact
artifact = wandb.Artifact(args["train_target_artifact"],
                          type="TRAIN_DATA",
                          description="A json file representing the train_y"
                          )

logger.info("Logging train_y artifact")
artifact.add_file(args["train_target_artifact"])
run.log_artifact(artifact)

16-10-2022 15:10:14 Logging train_y artifact


In [13]:
# test_x artifact
artifact = wandb.Artifact(args["test_feature_artifact"],
                          type="TEST_DATA",
                          description="A json file representing the test_x"
                          )

logger.info("Logging test_x artifact")
artifact.add_file(args["test_feature_artifact"])
run.log_artifact(artifact)

16-10-2022 15:10:49 Logging test_x artifact


In [14]:
# test_y artifact
artifact = wandb.Artifact(args["test_target_artifact"],
                          type="TEST_DATA",
                          description="A json file representing the test_y"
                          )

logger.info("Logging test_y artifact")
artifact.add_file(args["test_target_artifact"])
run.log_artifact(artifact)

16-10-2022 15:10:56 Logging test_y artifact


In [15]:
run.finish()